In [1]:
# store list provided from Margaret today
# RE: BL Sealy Zip List needed
# 3:22 pm
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

print(datetime.datetime.now())
print(os.getcwd())

2020-03-05 16:53:11.417426
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/Margaret_zips_11Markets_20200305


In [2]:
zip_dma=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
zip_dma=zip_dma.iloc[:,[0,2]]
zip_dma.columns=['zip_cd',"DMA"]
zip_dma=zip_dma.drop_duplicates()
print(zip_dma['zip_cd'].apply(len).unique())
zip_dma.head(4)

[5]


,zip_cd,DMA
0,00501,NEW YORK
1,00544,NEW YORK
2,01001,SPRINGFIELD-HOLYOKE
3,01002,SPRINGFIELD-HOLYOKE


In [3]:
df_store_list=pd.read_excel("./Copy of Top Mattress markets.xlsx",sheet_name="Sheet1",dtype=str)
list_190_stores=df_store_list['Store Id'].unique().tolist()
len(list_190_stores)

190

In [4]:
list_11_markets=df_store_list['Market'].unique().tolist()
len(list_11_markets)

11

In [5]:
zip_dma[zip_dma['DMA'].isin(list_11_markets)]['DMA'].nunique()

9

In [6]:
[x for x in list_11_markets if x not in zip_dma['DMA'].unique().tolist()]

['GREENSBORO-H.POINT-W.SALE', 'ORLANDO-DAYTONA BCH-MELBR']

In [7]:
zip_dma['DMA']=zip_dma['DMA'].replace("GREENSBORO-H.POINT-W.SALEM","GREENSBORO-H.POINT-W.SALE")
zip_dma['DMA']=zip_dma['DMA'].replace("ORLANDO-DAYTONA BCH-MELBRN","ORLANDO-DAYTONA BCH-MELBR")

In [8]:
[x for x in list_11_markets if x not in zip_dma['DMA'].unique().tolist()]

[]

In [9]:
# checked all the 190 stores are in the March store list

df_store_info=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",sep="|",dtype=str)
df_store_info=df_store_info[df_store_info['location_id'].isin(list_190_stores)]
print(df_store_info.shape,df_store_info['location_id'].nunique())
df_store_location=df_store_info[['location_id','latitude_meas','longitude_meas']]
df_store_location['latitude_meas']=df_store_location['latitude_meas'].astype(float)
df_store_location['longitude_meas']=df_store_location['longitude_meas'].astype(float)

df_store_location.head(4)

(190, 13) 190


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,location_id,latitude_meas,longitude_meas
2,42,38.740250,-82.965700
12,134,39.447216,-83.805895
15,157,40.968469,-80.308930
17,198,36.957020,-84.072252


In [10]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [11]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [12]:
df_zip_by_store=df_zip_by_store[df_zip_by_store['location_id'].isin(list_190_stores)]
print(df_zip_by_store['location_id'].nunique())

182


In [13]:
df_store_190_output=pd.merge(df_store_location,df_zip_by_store,on="location_id",how="left")

In [14]:
df_store_190_output_new=df_store_190_output[pd.isnull(df_store_190_output['zip_type'])]
df_store_190_output_existing=df_store_190_output[pd.notnull(df_store_190_output['zip_type'])]

df_store_190_output_new.shape,df_store_190_output_existing.shape

((8, 5), (3147, 5))

In [15]:
df_store_190_output_existing['zip_type'].unique()

array(['P', 'S', 'zip_10'], dtype=object)

In [16]:
del df_store_190_output_new['zip_cd']
del df_store_190_output_new['zip_type']
df_new_store_zip=pd.DataFrame()
for ind,row in df_store_190_output_new.iterrows():
    store=row['location_id']
    store_centroid=(row['latitude_meas'],row['longitude_meas'])
    list_store_zip=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(v,store_centroid,unit="mi")
        if dist<=10:
            list_store_zip.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_store_zip})
    df['zip_type']="zip_10"
    df['location_id']=store
    df_new_store_zip=df_new_store_zip.append(df)
    

In [17]:
df_store_190_output_new=pd.merge(df_store_190_output_new,df_new_store_zip,on="location_id",how="left")
df_store_190_output=df_store_190_output_existing.append(df_store_190_output_new)
df_store_190_output=df_store_190_output.rename(columns={"location_id":"Store Id"})

In [19]:
df_store_list=pd.merge(df_store_list,df_store_190_output,on="Store Id",how="left")

In [20]:
df_store_list.shape


(3345, 13)

In [22]:
df_all_zips_in_11_mkts=zip_dma[zip_dma['DMA'].isin(df_store_list['Market'].unique()).tolist()]
print(df_all_zips_in_11_mkts.shape,df_all_zips_in_11_mkts['zip_cd'].nunique(),df_all_zips_in_11_mkts['DMA'].nunique())


(3947, 2) 3919 11


In [24]:
writer=pd.ExcelWriter("./BL_zips_for_190_Stores_in_11_Markets_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_unique_zips_190=df_store_list[['zip_cd']].drop_duplicates()
df_unique_zips_190.to_excel(writer,"unique_zips_for_190_stores",index=False)

df_unique_zips_with_Mkt=df_store_list[['zip_cd','Market']].drop_duplicates()
df_unique_zips_with_Mkt.to_excel(writer,"zips_by_11_mkts",index=False)

df_all_zips_in_11_mkts.to_excel(writer,"all_zips_11_mkts",index=False)

df_store_list.to_excel(writer,"detail",index=False)

writer.save()